<!-- PROFILE LINK -->

<h1 style = "text-align:center; font-size: 30px">Actividad Individual 1</h1>
<a href ="https://github.com/jafethsuarez">
<img src="https://avatars.githubusercontent.com/u/44213172?v=4" style="float:left;vertical-align:centre" width="110" height="110" title = "Marcos Jafeth Azabache Suarez 22251035">
<a href ="https://github.com/sukuzhanay?tab=repositories">
<img src="https://media-exp1.licdn.com/dms/image/C4D0BAQEu_Aa76fcmPw/company-logo_200_200/0/1630998679429?e=2159024400&v=beta&t=2feC9GG7RWCWizTqsK5HzDWB1TL2DOQuAVwnMf6FFvk" title = "M41" alt="UE" width="110" height="110" style="float:right;vertical-align:centre">

#### Enlace a repositorio:  https://github.com/jafethsuarez/Prograconcurrente--> u22251035AI1_Servidor


In [ ]:
import socket
import threading
import sys
import pickle
import os
import re as reg

class Servidor(): ##se hace una clase para llamarla como el main en c

	def __init__(self, host=socket.gethostname(), port=int(input("Que puerto quiere usar ? "))): ## saber el puerto e ip del host
		self.clientes = []
		self.nicknames = []
		print('\nSu IP actual es : ',socket.gethostbyname(host))
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(), '\n\tTotal Hilos activos en este punto del programa =', threading.active_count())
		self.s = socket.socket()
		self.s.bind((str(host), int(port))) ## enlazar el host y el puerto
		self.s.listen(30)                   ## espera a que conexiones entren
		self.s.setblocking(False)

		threading.Thread(target=self.aceptarC, daemon=True).start()   ## evitar race condition
		threading.Thread(target=self.procesarC, daemon=True).start()

		while True:                                                ## si le doy a 1 apago el servidor
			msg = input('\n << SALIR = 1 >> \n')
			if msg == '1':
				print(" **** Me piro vampiro; cierro socket y mato SERVER con PID = ", os.getpid())
				self.s.close()
				sys.exit()
			else: pass

	def aceptarC(self):
		print('\nHilo ACEPTAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		
		while True:
			try:
				conn, addr = self.s.accept()
				print(f"\nConexion aceptada via {addr}\n")
				conn.setblocking(False)
				self.clientes.append(conn)
			except: pass

	def procesarC(self):
		print('\nHilo PROCESAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())  
		while True:
			if len(self.clientes) > 0:   ##si no hay clientes conectados no se conectan 
				for c in self.clientes:
					try:
						data = c.recv(64)                   ##esperoen caso este alguien  en el broadcast
						if data: self.broadcast(data,c)
					except: pass

	def broadcast(self, msg, cliente):        ##envia el mensaje a todos los clientes conectados
		str=pickle.loads(msg)
		found=reg.search('^[^ :]*',str)
		if found.group() not in self.nicknames:
			self.nicknames.append(found.group())
		for c in self.clientes:               ## visualizamos cuantos clientes hay conectados
			print("Clientes conectados Right now = ", len(self.clientes))
			print("LISTA DE CLIENTES (NICKNAMES)")
			for n in self.nicknames:
				print(n)
			try:
				if c != cliente:             
					print(pickle.loads(msg))
					c.send(msg)
			except: self.clientes.remove(c)

arrancar = Servidor() 

# Evidencia en el PDF